In [1]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [2]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk", "data/osm/ZMG_enclosure_2km.geojson")


Loading graph from data/graphs/ZMG_walk.pkl


In [3]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)


In [ ]:
def create_graph_transit(transit_df, stops_df):
    # create graph transit adjacency list in stops_df
    G = nx.MultiDiGraph()
    
    #to have the geometry in meters
    stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)

    for idx, stop in stops_gdf.iterrows():
        stop_id = stop['stop_id']
        G.add_node(stop_id, pos=stop['geometry'], stop_name=stop['stop_name'], routes=stop['routes_by_stop'], shapes=stop['shapes_by_stop'])
        next_stops = stop['next_stop_id']
        for next_stop_id, edge_list in next_stops.items():
            for edge in edge_list:                
                G.add_edge(stop_id, next_stop_id, weight=edge['weight'], shape_id=edge['shape_id'], frequency=edge['frequency'])
    return G

def add_walking_edges(graph_transit:pd.DataFrame, stops_df:pd.DataFrame, max_walking_time=300):
    walking_speed_mps = 5 / 3.6 # average walking speed 5 km/h in m/s
    walking_distance = max_walking_time * walking_speed_mps  

    # to have the geometry in meters
    stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)

    # Use spatial index to efficiently find nearby stops
    sindex = stops_gdf.sindex

    for stop_idx, stop in stops_gdf.iterrows():
        stop_id = stop['stop_id']
        stop_point_geo = stop['geometry']
        stop_circle_walking = stop_point_geo.buffer(walking_distance)

        # candidate indices whose geometries intersect the walking reach
        nearby_stops_idx = sindex.query(stop_circle_walking, predicate="intersects")
        if len(nearby_stops_idx) == 0:
            continue

        for nearby_stop_idx in nearby_stops_idx:
            # we dont want to add walking edge to itself
            if nearby_stop_idx == stop_idx:
                continue
            nearby_stop = stops_gdf.iloc[nearby_stop_idx]
            #calculate real distance in meters between the two stops
            distance = stop_point_geo.distance(nearby_stop['geometry'])            
            # walking time in seconds
            walking_time = round(distance / walking_speed_mps) 
            # add walking edge, shape_id='walking' due to the mode of transport, frequency=0 as its not a scheduled transport you have to wait
            graph_transit.add_edge(stop_id, nearby_stop['stop_id'], weight=walking_time, shape_id='walking', frequency=0)
        
graph_transit = create_graph_transit(transit_df, stops_df)
print(graph_transit)
add_walking_edges(graph_transit, stops_df, max_walking_time=300)
print(graph_transit)

   

MultiDiGraph with 10345 nodes and 23871 edges
MultiDiGraph with 10345 nodes and 169475 edges
MultiDiGraph with 10345 nodes and 169475 edges


In [ ]:
#get first two nodes in the graph_transit
start = list(graph_transit.nodes)[0]
destination = list(graph_transit.nodes)[1]
#measure distance between start and destination
print(graph_transit.nodes[start]['pos'].distance(graph_transit.nodes[destination]['pos']))

322.72307548412056


In [ ]:
import heapq
from shapely import Point, LineString

#src and dst are Point in meters, crs EPSG:3857
def euclidean_heuristic(src:Point, dst:Point) -> float:
    # Time to reache the dst
    # distance in meters using euclidean distance
    transit_average_speed_kph = 55.0  # average transit max speed in km/h
    transit_average_speed_mps = transit_average_speed_kph / 3.6  # convert to m/s
    distance_meters = src.distance(dst)
    time_to_reach = distance_meters / transit_average_speed_mps  #time in s
    return round(time_to_reach)

def no_heuristic(src, dst):
    return 0
 

def check_no_transfers(graph:nx.MultiDiGraph, src, dst, transit_df, stops_df):

     #verify if src and dst share a shape, only one bus is taken
    src_shapes = set(stops_df[stops_df['stop_id'] == src]['shapes_by_stop'].iloc[0])
    dst_shapes = set(stops_df[stops_df['stop_id'] == dst]['shapes_by_stop'].iloc[0])

    shared_shapes = src_shapes & dst_shapes

    if shared_shapes:
        print("Shared shapes found:", shared_shapes)
        shape = shared_shapes.pop()
        shape_info = transit_df[transit_df['shape_id'] == shape].iloc[0]
        
        shape_stops = shape_info['stop_ids']
        src_index = shape_stops.index(src)
        dst_index = shape_stops.index(dst)
        if src_index < dst_index:
            path = shape_stops[src_index:dst_index + 1]
            stop_time_deltas = shape_info['stop_time_deltas']          
            total_cost = sum(stop_time_deltas[src_index:dst_index])
            
            #add to the path the shape info
            path = [(stop, shape) for stop in path]
            return path, total_cost
        
    return [], None

def check_one_transfer(graph:nx.MultiDiGraph, src, dst, transit_df, stops_df):
    
        #verif if src and dst shapes share a stop, only one transfer is needed
    src_shapes = set(stops_df[stops_df['stop_id'] == src]['shapes_by_stop'].iloc[0])
    dst_shapes = set(stops_df[stops_df['stop_id'] == dst]['shapes_by_stop'].iloc[0])

    for src_shape in src_shapes:
        src_shape_stops = transit_df[transit_df['shape_id'] == src_shape]['stop_ids'].iloc[0]
        for dst_shape in dst_shapes:
            dst_shape_stops = transit_df[transit_df['shape_id'] == dst_shape]['stop_ids'].iloc[0]
            #find common stops
            common_stops = set(src_shape_stops) & set(dst_shape_stops)
            if common_stops:
                transfer_stop = common_stops.pop()
                #build path from src to transfer_stop
                src_index = src_shape_stops.index(src)
                transfer_index_src = src_shape_stops.index(transfer_stop)
                path_src = src_shape_stops[src_index:transfer_index_src + 1]
                #build path from transfer_stop to dst
                transfer_index_dst = dst_shape_stops.index(transfer_stop)
                dst_index = dst_shape_stops.index(dst)
                path_dst = dst_shape_stops[transfer_index_dst:dst_index + 1]
                #combine paths
                full_path = path_src + path_dst[1:]  #avoid duplicating transfer_stop
                total_cost = 0
                for i in range(len(full_path) - 1):
                    edge_data = graph.get_edge_data(full_path[i], full_path[i + 1])
                    weight = edge_data.get('weight', 1)
                    total_cost += weight
                return full_path, total_cost    
    
    return [], None

def dijkstra(graph:nx.MultiDiGraph, src, dst, transit_df: pd.DataFrame, stops_df: pd.DataFrame, heuristic=euclidean_heuristic):
    #accumulated cost for each node from the start, keys are (node, shape_id)
    cost = dict()
    #stores the node and shape that it was coming from, keys are (node, shape_id)
    previous = dict()

    #"minimum heap" of the candidates (priorty_cost, node, shape_id)
    queue = []
    #to return the path from src to dst, list of tuples (node, shape_id)
    path = []
    
    #path, total_cost = check_no_transfers(graph, src, dst, transit_df, stops_df)
    if path:
        print("No transfers needed.")
        return path, total_cost
    #path, total_cost = check_one_transfer(graph, src, dst, transit_df, stops_df)
    if path:
        print("One transfer needed.")
        return path, total_cost      

    # fill the queue with all the possible starting edges from src
    for neighbor in graph.neighbors(src):
        edges = graph.get_edge_data(src, neighbor)
        if not edges:
            continue
        for k, attrs in edges.items():
            shape_id = attrs.get('shape_id')
            weight = attrs.get('weight', 1)
            frequency = attrs.get('frequency', 600)
            tentative_cost = weight + frequency #initial transfer penalty
            cost[(neighbor, shape_id)] = tentative_cost
            previous[(neighbor, shape_id)] = (src, None)
            heuristic_cost = heuristic(graph.nodes[neighbor]['pos'], graph.nodes[dst]['pos'])
            priority_cost = tentative_cost + heuristic_cost
            heapq.heappush(queue, (priority_cost, neighbor, shape_id))
    

    end_shape = None

    while queue:

        _, current, current_shape = heapq.heappop(queue)

        if current == dst:
            end_shape = current_shape
            break


        # explore all neighbors (MultiDiGraph may have parallel edges) and use min edge weight
        for neighbor in graph.neighbors(current):

            edges = graph.get_edge_data(current, neighbor)
            if not edges:
                continue
            
            for k, attrs in edges.items():
                next_shape = attrs.get('shape_id')
                weight = attrs.get('weight', 1)
                frequency = attrs.get('frequency', 600)
                
                if (current_shape is not None and next_shape is not None and next_shape != current_shape):
                    penalty = frequency
                else:
                    penalty = 0
                
                tentative_cost = cost[(current, current_shape)] + weight + penalty

                # if cost for neighbor with next_shape is not set or tentative_cost is lower, update
                if (neighbor, next_shape) not in cost or tentative_cost < cost[(neighbor, next_shape)]:
                    cost[(neighbor, next_shape)] = tentative_cost                
                    previous[(neighbor, next_shape)] = (current, current_shape)
                    heuristic_cost = heuristic(graph.nodes[neighbor]['pos'], graph.nodes[dst]['pos']) + 3*penalty
                    priority_cost = tentative_cost + heuristic_cost
                    heapq.heappush(queue, (priority_cost, neighbor, next_shape))
        
    if end_shape is None:
        print("No path found from", src, "to", dst)
        return [], None


    #recreate the path based on the previous going backwards but store it in forward 
    current = (dst, end_shape)

    while current in previous:
        path.append(current)
        current = previous.get(current)
    path.reverse()
    
    return path, cost[(dst, end_shape)]


In [ ]:
#choose randomly a node from G
import random

path = []

while path == []:
    start = random.choice(list(graph_transit.nodes))
    destination = random.choice(list(graph_transit.nodes))

    try:
        path = nx.shortest_path(graph_transit, source=start, target=destination, weight='weight')
    except nx.NetworkXNoPath:
        path = []

print("Shortest path from", start, "to", destination, ":", path)


Shortest path from mxc_C27_STP_45 to LM-V02_r1_28 : ['mxc_C27_STP_45', 'MM_C27_45', 'MP-C-044', 'MM_C21_02', 'mxc_C108V1_STP_26', 'mxc_C108V1_STP_27', 'mxc_C108V1_STP_28', 'mxc_C68_STP_61', 'mxd_mxc_AMG_T18_1_STP_60', 'mxd_mxc_AMG_T18_1_STP_61', 'mxd_mxc_AMG_T18_1_STP_62', 'mxd_mxc_AMG_T18_1_STP_63', 'mxc_C08_STP_32', 'mxd_mxc_AMG_T18_1_STP_65', 'mxd_mxc_AMG_T18_1_STP_66', 'mxc_C69_STP_39', 'mxd_mxc_AMG_T18_1_STP_68', 'mxd_mxc_AMG_T18_1_STP_69', 'mxd_mxc_AMG_T18_1_STP_70', 'mxd_mxc_AMG_T18_1_STP_71', 'mxd_mxc_AMG_T18_1_STP_72', 'mxc_C38V1_STP_56', 'mxc_C38V1_STP_57', 'mxc_C97V2_STP_67', 'mxd_mxc_AMG_T18_1_STP_76', 'mxc_C97V2_STP_69', 'mxc_C97V2_STP_70', 'mxd_mxc_AMG_T18_1_STP_79', 'mxc_C118_STP_62', 'mxc_C125V1_STP_97', 'mxc_C129_STP_90', 'TL_0001_0010', 'TL_0001_0009', 'MT-L1_09', 'TL_0001_0008', 'MT-L1_08', 'MT-L1_07', 'TL_0001_0007', 'TL_0001_0006', 'MT-L1_06', 'MT-L1_05', 'TL_0001_0005', 'MT-L1_04', 'TL_0001_0004', 'TL_0001_0003', 'MT-L1_03', 'TL_0001_0002', 'MT-L1_02', 'MT-L1_01',

In [ ]:
dijkstra_path, dijkstra_cost = dijkstra(graph_transit, start, destination, transit_df, stops_df, heuristic=euclidean_heuristic)
print("Shortest path from", start, "to", destination, ":", dijkstra_path)

print("Cost of the path:", dijkstra_cost/60)

transit_gdf = gpd.GeoDataFrame(transit_df, geometry='shape_geometry', crs='EPSG:4326')

dijkstra_path_stops = []
dijkstra_path_shapes = []
prev_shape = None
for stop, shape in dijkstra_path:
    dijkstra_path_stops.append(stop)
    dijkstra_path_shapes.append(shape)
    if prev_shape is None:
        print("Walk to stop:", stop)
    if shape == 'walking':
        print("Walk to stop:", stop)
    elif shape != prev_shape:
        print("Take Bus:", shape, "from stop:", stop, " direction to:", transit_df[transit_df['shape_id'] == shape]['trip_headsign'].iloc[0])
    prev_shape = shape


#show all the shapes in the path
#assign colors based on shape_id or randomly
shapes_in_path = sorted(set(dijkstra_path_shapes))
subset = transit_gdf[transit_gdf['shape_id'].isin(shapes_in_path)]
m = subset.explore(
    column='route_short_name',
    cmap='tab20',
    legend=True,
    tooltip=['route_long_name', 'route_short_name', 'trip_headsign'],
    style_kwds={'weight': 6, 'opacity': 0.9}
)
print(set(shapes_in_path))

stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)

path_gdf = stops_gdf[stops_gdf['stop_id'].isin(dijkstra_path_stops)]
m = path_gdf.explore(color='red', m=m)
openMap(m)

Shortest path from mxc_C27_STP_45 to LM-V02_r1_28 : [('mxc_C27_STP_46', 'C28_r1'), ('mxc_C27_STP_47', 'C28_r1'), ('mxc_C27_STP_48', 'C28_r1'), ('mxc_C27_STP_49', 'C28_r1'), ('mxc_C123V1_STP_14', 'C28_r1'), ('mxc_C123V1_STP_15', 'C28_r1'), ('mxc_C123V1_STP_16', 'C28_r1'), ('mxc_C123V1_STP_17', 'C28_r1'), ('mxc_C123V1_STP_18', 'C28_r1'), ('mxc_C123V1_STP_19', 'C28_r1'), ('mxc_C13V2_STP_31', 'walking'), ('mxd_mxc_AMG_T18_1_STP_60', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_61', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_62', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_63', 'T18-1-T_r1'), ('mxc_C08_STP_32', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_65', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_66', 'T18-1-T_r1'), ('mxc_C69_STP_39', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_68', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_69', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_70', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_71', 'T18-1-T_r1'), ('mxd_mxc_AMG_T18_1_STP_72', 'T18-1-T_r1'), ('mxc_C38V1_STP_56', 'T18-1-T_r1'), ('mxc_C3

In [ ]:
# To have the geometry (Points) in meters for distance calculations
stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)
stops_gdf['circles'] = stops_gdf.geometry.buffer(100)  # 50 meters buffer around each stop

#check every stop circle if it intersects with any other stop point
for idx, stop in stops_gdf.iterrows():
    stop_id = stop['stop_id']
    circle = stop['circles']
    intersecting_stops = stops_gdf[stops_gdf.geometry.within(circle) & (stops_gdf['stop_id'] != stop_id)]
    if not intersecting_stops.empty:
        print("Stop", stop_id, "intersects with stops:", intersecting_stops['stop_id'].tolist())

#change geometry to circles
stops_gdf.set_geometry('circles', inplace=True)
m = stops_gdf.explore(color='blue', opacity=0.05)
stops_gdf.set_geometry('geometry', inplace=True)
m = stops_gdf.explore(color='red', m=m)
openMap(m)


Stop mxa_T06_03_STP_19 intersects with stops: ['MM_T19C03_149']
Stop mxa_T06_03_STP_20 intersects with stops: ['mxc_C07V1_STP_34']
Stop mxa_T06_03_STP_21 intersects with stops: ['mxc_C07V1_STP_35']
Stop mxa_T06_03_STP_26 intersects with stops: ['mxc_C07V1_STP_40']
Stop mxa_T06_03_STP_27 intersects with stops: ['mxc_C07V1_STP_41']
Stop mxa_T06_03_STP_28 intersects with stops: ['mxa_T13C1_STP_62', 'mxc_C08_STP_39']
Stop mxa_T06_03_STP_36 intersects with stops: ['mxc_C66V1_STP_38']
Stop mxa_T06_03_STP_49 intersects with stops: ['mxc_C22_STP_74']
Stop mxa_T06_03_STP_50 intersects with stops: ['mxc_C08_STP_59', 'mxd_mxc_AMG_T18_2_STP_24', 'MM_T18_2_24']
Stop mxa_T06_03_STP_52 intersects with stops: ['mxc_C07V1_STP_71', 'MP-C-076']
Stop mxa_T06_03_STP_55 intersects with stops: ['mxd_mxc_AMG_T02_STP_31']
Stop mxa_T06_03_STP_58 intersects with stops: ['mxc_C07V1_STP_77', 'mxc_C75_STP_11', 'mxc_C75_STP_70', 'mxd_mxc_AMG_T09C01_STP_141']
Stop mxa_T06_03_STP_69 intersects with stops: ['mxc_C108V1